In [1]:
import pysolr
from Solr.create_doc import Create_doc
from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems

In [6]:
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":32,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-103"}},\n  "status":"OK"}\n'

In [7]:
def UpadateSolr(path_marc, count, shelf):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #SOLR
        doc = Create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)
        
        count += 1
    return docs, count

In [ ]:
count = 686
e = '5'
p = '5'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs, _ = UpadateSolr(path_marc, count, f"E{e}.P{p}")

In [14]:
solr.add(docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":175}}\n'

In [ ]:
count = 1
solr_doc = list()
for e in range(1, 6):
    for p in range(1,6):
        out = f'E{e}/P{p}'
        path_marc = f'Input/{out}/koha.xml'
        docs, count = UpadateSolr(path_marc, count, f"E{e}.P{p}")
        solr_doc = solr_doc+docs




In [10]:
solr.add(solr_doc, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":3676}}\n'

In [2]:
out = f'E1/P1'
count = 1
path_marc = f'Input/{out}/koha.xml'
marc_file = parse(path_marc)
records = marc_file.getElementsByTagName('record')
record = records[0]
marcxml = etree.fromstring(record.toxml())
workMarc = MarcWork(marcxml)
instanceMarc = MarcInstance(marcxml)
itemsMarc = MarcItems(marcxml)
shelf = 'E1.P1'


In [3]:
serie = instanceMarc.Serie()
serie

{'serie': 'Pensamento científico', 'volume': 'n. 1'}

In [4]:
doc = Create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)

In [5]:
doc

{'id': 1,
 'call': '001 P831c',
 'shelf': 'E1.P1',
 'title': 'Conjecturas e refutações',
 'responsibilities': 'Karl R. Popper; tradução de Sérgio Bath.',
 'place': 'Brasília',
 'publisher': 'Ed. Universidade de Brasília',
 'subject': ['Ciência--Metodologia', 'Teoria do conhecimento'],
 'type': 'Livro',
 'language': 'por',
 'items': ['01-0519'],
 'author': 'Popper, Karl Raimund',
 'serie': 'Pensamento científico',
 'numSerie': 'n. 1',
 'year': '1982',
 'notes': ['Título original: Conjectures and refutations, 4th ed. rev, 1972.'],
 'pages': '449 páginas'}